# Running

To run the Traffic Sign's detector model, one of the ways of doing that is using Nvidia SDK: DeepStream [[Official documentation here](https://developer.nvidia.com/deepstream-sdk)], so follow the steps below:
### 1. Getting the Docker Container
**Attention:** To make use of the detector with your own Webcam, you must disable all host running this command:
```console
xhost +
```
We will use a Docker Container to get in touch with the DeepStream Toolkit. For this project (specifically), a modified version of the Docker image was posted [[here](https://hub.docker.com/r/edgardaon/nvidia-deepstream-6.0-triton)] aiming to spare the user from the need of software configuration and installation. Run the command down below to obtain a *bash* with all the project infrastructure:  
```console
docker run --gpus all -it --rm --net=host --privileged -v /tmp/.X11-unix:/tmp/.X11-unix -w /opt/nvidia/deepstream/deepstream-6.0 edgardaon/nvidia-deepstream-6.0-triton:projetoML
```

**Attention:** However, if you don't want this version but do want to do a fresh configuration of the SDK all by yourself, feel free to follow the steps from the Nvidia NGC: [https://catalog.ngc.nvidia.com/orgs/nvidia/containers/deepstream](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/deepstream) After that, you should clone the [NVIDIA-IOT's DeepStream Python Apps Repository](https://github.com/NVIDIA-AI-IOT/deepstream_python_apps) to make use of python along with DeepStream.

### 2. Inside the Docker Container
Inside the container, run the command: ***deepstream-app --version-all***. If everything is set, you should see this versions in the output below:


Software  | Version
------------- | -------------
deepstream-app  | 6.0.0
DeepStreamSDK  | 6.0.0 
CUDA Driver  | 11.7
CUDA Runtime  | 11.4
TensorRT   | 8.0
cuDNN   | 8.2
libNVWarp360   | 2.0.1d3

Then, go to the directory where the project files are located, which is ***/opt/nvidia/deepstream/deepstream-6.0/sources/deepstream_python_apps/apps/deepstream-projeto/***. There you shold see the python script (*deepstream_test_1_usb.py*), the DeepStream configuration file (*dstest1_pgie_config.txt*), the model labels (*labels.txt*), the detector encoded model (*resnet18_detector.etlt*) (*.etlt* extension stands for encoded transfer learning toolkit, a file created by the TAO Toolkit), the detector engine (*resnet18_int8_detector.engine*) and the last, but not least, calibration file (*calibration.bin*).


Before running the detector, it is important to check Deepstream configuration file:
```text
[property]
gpu-id=0
net-scale-factor=0.0039215697906911373
model-engine-file=./resnet18_int8_detector.engine
int8-calib-file=./calibration.bin
labelfile-path=./labels.txt
tlt-encoded-model=./resnet18_detector.etlt
tlt-model-key=roboflow
infer-dims=3;640;640
uff-input-order=0
uff-input-blob-name=input_1
batch-size=1
process-mode=1
model-color-format=0
## 0=FP32, 1=INT8, 2=FP16 mode
network-mode=1
num-detected-classes=3
interval=0
gie-unique-id=1
output-blob-names=output_bbox/BiasAdd;output_cov/Sigmoid

[class-attrs-all]
pre-cluster-threshold=0.2
eps=0.2
group-threshold=1

```
**Note**: If you haven't used the *edgardaon/nvidia-deepstream-6.0-triton* docker image (referenced in the docker run command above) and is running another one, like the default from *NGC (Nvidia GPU Cloud) Catalog*, you should upload the *deepstream_test_1_usb.py* project python script to a any folder as long as it has the others TAO exported files also.

There are some properties that are worth noting, like *gpu_id* ensure us that the model will run at the main GPU, the *model-engine-file* needs to be the exact detector *.engine* file and the same goes for the *int8-calib-file*, *labelfile-path*, *tlt-encoded-model* and *tlt-model-key*, receiving the *calibration.bin*, *labels.txt*, *resnet18_detector.etlt* files with *roboflow* password. After that, there's the *num-detected-classes* which are going to be 3 since this is the number types of signs we have to detect: *stop* ("pare"), *40* and *60*.
After configuring these, we can run the detector using the command below, where */dev/video0* is the webcam address for this example:
```console
python3 deepstream_test_1_usb.py /dev/video0
```
Running this code, you should see in the upper left corner a frame, a sign total, stop sign, 40 sign and 60 sign counters with the detections (if they may occur) on screen. Below, we have a demonstration of the model detecting stop signs, 40 signs and 60 signs, respectively.


#### Demonstrations:
<div align="center">
    <img src="../Videos/placaPARE.gif" />
    <img src="../Videos/placa40.gif" />
    <img src="../Videos/placa60.gif" />
</div>